In [2]:
%endpoint http://labo-vm-2.v202.abes.fr:7200/repositories/men-habilitations-v3-habilitations-partielles-inference

Endpoint set to: http://labo-vm-2.v202.abes.fr:7200/repositories/men-habilitations-v3-habilitations-partielles-inference

<center><h1>Movies</h1></center>

---

## Modélisation des habilitations doctorales (test)

### Habilitation simple

<center>
    <figure>
        <img width="90%" src="habilitation_doctorale.svg">
        <figcaption>Modélisation de l'habilitation doctorale</figcaption>
    </figure>
</center>

### Délégation de l'habilitation doctorale

<center>
    <figure>
        <img width="90%" src="delegation_doctorale.svg">
        <figcaption>Modélisation de la délégation doctorale</figcaption>
    </figure>
</center>

### Délégations et habilitations partielles

<center>
    <figure>
        <img width="90%" src="habilitations_doctorales_partielles.svg">
        <figcaption>Modélisation de l'habilitation partielle et de la délégation partielle</figcaption>
    </figure>
</center>

__États des établissements__

En matière de compétences doctorales il existe quatre états distincts :

* `habilitation` l'établissement possède l'intégralité de l'habilitation
* `delegationDoctorale` l'établissement délègue l'intégralité de son habilitation à un établissement tiers
* `habilitationPartielle` l'établissement est habilité pour une partie des écoles doctorales seulement (implique une délégation partielle)
* `delegatioPartielle` l'établissement délègue son habilitation pour une partie des écoles doctorales seulement (implique une habilitation partielle)

Quelques règles régissent les états doctoraux : ils sont exclusifs les uns des autres ; ils sont bornés chronologiquement, ces bornes doivent s'inscrire dans celles de l'établissement auquel ils se rapportent : une habilitation ne peut pas être antérieure à la création d'un établissement ou postérieure à la fermeture d'un établissement ; l'état de délégation partielle implique nécessairement l'existence d'un état d'habilitation partielle.

__Propriétés :__

| nom        | description             | obligatoire                     | note |
|------------|-------------------------|---------------------------------|------|
| `from`     | établissement source    | <input type="checkbox" checked> |      |
| `to`       | établissement cible     | <input type="checkbox">         |      |
| `start`    | date de début de l'état | <input type="checkbox" checked> |      |
| `end`      | date de fin de l'état   | <input type="checkbox">         |      |
| `notes`    |                         | <input type="checkbox">         |   N'est utilisée que dans le cadre de la délgation partielle et l'habilitation partielle. Cette  note précise quelles sont les écoles doctorales concernées |



## Cas d'usage

### Liste des habilitations doctorales

> un établissement peut être habilité plusieurs fois

In [3]:
%qparam infer false
%qparam sameAs false
%show 20

PREFIX mov: <http://www.abes.fr/vocabularies/movies#>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX mov_state: <http://www.abes.fr/entities/>
        
SELECT ?statetype ?start ?end ?codeEtab ?from ?to WHERE {
    ?etat a mov_state:etats;
        mov:from ?from; mov:start ?start; mov:statetype ?statetype.
    
    OPTIONAL {
        ?etat mov:end ?end.
    }
    
    OPTIONAL {
        ?etat mov:to ?to.
    }
    
    ?from mov:codeEtabid ?codeEtab; foaf:name ?etab.
    
    FILTER(?statetype IN ("habilitationDoctorale", "delegationDoctorale", "habilitationPartielle", "delegationPartielle"))
}

Param set: infer = false
Param set: sameAs = false
Result maximum size: 20

Error: Query processing error: <urlopen error [Errno -2] Name or service not known>

### Liste des établissements habilités

In [6]:
%qparam infer false
%qparam sameAs false
%show 20

PREFIX mov: <http://www.abes.fr/vocabularies/movies#>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>       
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX mov_state: <http://www.abes.fr/entities/>

SELECT ?from ?codeEtab ?name WHERE {
  {
    SELECT ?from WHERE {
        ?etat a mov_state:etats;
        mov:from ?from; mov:start ?start; mov:statetype ?statetype.
    
        OPTIONAL {
            ?etat mov:end ?end.
        }
        
        FILTER(?statetype IN ("habilitationDoctorale", "habilitationPartielle"))
        FILTER (!BOUND(?end) || ?end > NOW())
    }
    GROUPBY(?from)
  }
   ?from mov:codeEtabid ?codeEtab; foaf:name ?name.
}

Param set: infer = false
Param set: sameAs = false
Result maximum size: 20

Error: Query processing error: <urlopen error [Errno -2] Name or service not known>

### Liste des établissements qui ne sont plus habilités

In [7]:
%qparam infer false
%qparam sameAs false
%show 20

PREFIX mov: <http://www.abes.fr/vocabularies/movies#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>       

SELECT ?from ?codeEtab ?name WHERE {
  {
    SELECT ?from WHERE {
        ?etat a <http://www.abes.fr/entities/etats>;
        mov:from ?from; mov:start ?start; mov:statetype ?statetype.
    
        ?etat mov:end ?end.
        
        FILTER(?statetype IN ("habilitationDoctorale", "habilitationPartielle"))
        FILTER (?end < NOW())
    }
    GROUPBY(?from)
  }
   ?from mov:codeEtabid ?codeEtab; foaf:name ?name.
}

Param set: infer = false
Param set: sameAs = false
Result maximum size: 20

Error: Query processing error: <urlopen error [Errno -2] Name or service not known>

### Filiation des environnements thèses

* Retrouver les filiations des habilitations doctorales à partir d'un établissement. Cela permet de regrouper les thèses d'une institution. Par exemple celles de Grenoble Alpes (expérimental) avec celle de Grenoble Alpes (Comue) etc.


In [4]:
%qparam infer true
%qparam sameAs true
%show 20

PREFIX paysage: <http://paysage.mesri.fr/structures/>
PREFIX mov: <http://www.abes.fr/vocabularies/movies#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

select ?x ?codeEtab ?xname ?state where {   
    BIND(paysage:Y7ch7 as ?grenobleAlpes)
    
    ?grenobleAlpes (^mov:to/mov:from|^mov:from/mov:to)* ?x.
    
    OPTIONAL {
       ?grenobleAlpes mov:codeEtabid ?codeEtab.    
    }
    
    ?x a foaf:Organization; skos:prefLabel ?xname;  mov:codeEtabid ?codeEtab.
    
    ?state mov:from ?x ; mov:statetype "existence".
    
    FILTER (strstarts(str(?x), 'http://paysage.mesri.fr/structures/'))
}

Param set: infer = true
Param set: sameAs = true
Result maximum size: 20

x,codeEtab,xname,state
http://paysage.mesri.fr/structures/hB4QE,GREA,Université Grenoble Alpes,http://www.abes.fr/entities/etat/existence/hB4QE/e011996ef62b1f7f01b7968c7af969e3
http://paysage.mesri.fr/structures/Mrr0h,GRE1,Université Grenoble I,http://www.abes.fr/entities/etat/existence/Mrr0h/8fb8db594330513b7b05ea4d062c0a1a
http://paysage.mesri.fr/structures/Mrr0h,GRE1,Joseph Fourier University,http://www.abes.fr/entities/etat/existence/Mrr0h/8fb8db594330513b7b05ea4d062c0a1a
http://paysage.mesri.fr/structures/VrYMi,GRE2,Université Grenoble-II,http://www.abes.fr/entities/etat/existence/VrYMi/88975f005c48b12498de3b60cf31a56a
http://paysage.mesri.fr/structures/VrYMi,GRE2,Pierre Mendès-France University,http://www.abes.fr/entities/etat/existence/VrYMi/88975f005c48b12498de3b60cf31a56a
http://paysage.mesri.fr/structures/XfV3j,GRE3,Université Grenoble-III,http://www.abes.fr/entities/etat/existence/XfV3j/06fb742dc2dec1323b31a44e98220a9e
http://paysage.mesri.fr/structures/XfV3j,GRE3,Stendhal University,http://www.abes.fr/entities/etat/existence/XfV3j/06fb742dc2dec1323b31a44e98220a9e
http://paysage.mesri.fr/structures/zCa4j,CHAM,Université de Chambéry,http://www.abes.fr/entities/etat/existence/zCa4j/355c932a0de742aa3760bc4a6adec681
http://paysage.mesri.fr/structures/zCa4j,CHAM,University of Savoy,http://www.abes.fr/entities/etat/existence/zCa4j/355c932a0de742aa3760bc4a6adec681
http://paysage.mesri.fr/structures/2ALYK,INPG,Institut polytechnique de Grenoble,http://www.abes.fr/entities/etat/existence/2ALYK/ad293659175ee280c1924086200f917c


## Contrôle qualité des données

### Validité des identifiants (PPN)

In [6]:
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

select * where { 
    ?ppn a foaf:Organization.
    
    FILTER (STRSTARTS(str(?ppn), "http://www.abes.fr/entities/organizations/ppn")) 
    FILTER (!regex(STR(?ppn), '/\\d{8}(\\d|X)', "i"))
}

ppn


### Validité des habilitations

* Seuls les établissements ayant une personnalité juridique peuvent délivrer le doctorat (que se passe-t-il s'ils changent de personnalité, cela implique-t-il une nouvelle habilitation ?)
* Les habilitations ne peuvent pas être antérieures ou postérieures à l'existence de l'établissement

In [95]:
%qparam infer true
%qparam sameAs false
%show 20

PREFIX mov: <http://www.abes.fr/vocabularies/movies#>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?codeEtab ?habilitation ?formeJuridique ?startHabilitation ?inception ?endHabilitation ?destruction ?diagnostic WHERE {
    {
        # On récupère les organisations qui sont ou on été habilités (c'est à dire qui ont un code établissement `?codeEtab`) ainsi que leur forme juridique.
        ?org a foaf:Organization;
             mov:codeEtabid ?codeEtab;
             wdt:P1454 ?formeJuridique.
        
        # On filtre les établissements qui ont explicitement une absence de personnalité juridique : `Sans personnalité juridique`.
        FILTER(?formeJuridique = "Sans personnalité juridique"@fr)
        
        # On assigne à la variable `?diagnostic` le contenu du message de diagnostic.
        BIND("Forme juridique invalide"@fr as ?diagnostic)
    } UNION {
        ?habilitation mov:from ?org ; mov:start ?startHabilitation ; mov:statetype ?state.
        
        FILTER(?state IN ("habilitationDoctorale", "delegationDoctorale", "delegationPartielle", "habilitationPartielle"))

        OPTIONAL {
            ?habilitation mov:end ?endHabilitation.
        }
        
        ?org mov:codeEtabid ?codeEtab.
        
        ?existence ?_ <http://www.abes.fr/entities/etats>;
                mov:from ?org;
                mov:statetype "existence".

        ?existence mov:start ?creation_etab.

        OPTIONAL {
            ?existence mov:end ?supression_etab.
        }
        
        # On filtre les établissements pour lesquels :
        # les dates d'habilitation dépassent les date d'existence
        # ou pour lesquels la fin de l'habilitation est antérieure au début de l'habilitation
        FILTER(?startHabilitation < ?inception || ?endHabilitation > ?destruction || ?endHabilitation < ?startHabilitation)
        BIND("Dates d'habilitation invalides"@fr as ?diagnostic)
    }
}

Param set: infer = true
Param set: sameAs = false
Result maximum size: 300

codeEtab,habilitation,formeJuridique,startHabilitation,inception,endHabilitation,destruction,diagnostic
EMAC,,Sans personnalité juridique,,,,,Forme juridique invalide
EMAL,,Sans personnalité juridique,,,,,Forme juridique invalide
EMSE,,Sans personnalité juridique,,,,,Forme juridique invalide
ENIS,,Sans personnalité juridique,,,,,Forme juridique invalide
ENST,,Sans personnalité juridique,,,,,Forme juridique invalide
IMTA,,Sans personnalité juridique,,,,,Forme juridique invalide
INPL,,Sans personnalité juridique,,,,,Forme juridique invalide
NSAM,,Sans personnalité juridique,,,,,Forme juridique invalide
TELB,,Sans personnalité juridique,,,,,Forme juridique invalide


### Établissements non liés à Paysage

In [7]:
%qparam infer false
%qparam sameAs false
%show 20

PREFIX mov: <http://www.abes.fr/vocabularies/movies#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT ?org_abes ?name WHERE {
    ?org_abes a foaf:Organization;
        foaf:name ?name.
    
    FILTER (STRSTARTS(str(?org_abes), "http://www.abes.fr/entities/organizations/ppn"))
    
    MINUS { 
        ?org_paysage (^owl:sameAs|owl:sameAs) ?org_abes .
        FILTER (STRSTARTS(str(?org_paysage), "http://paysage.mesri.fr/structures/"))
    }
    
    OPTIONAL {
        ?org_abes (^owl:sameAs|owl:sameAs) ?relation.    
    }
}

Param set: infer = false
Param set: sameAs = false
Result maximum size: 20

org_abes,name
http://www.abes.fr/entities/organizations/ppn/026402882,Aix-Marseille 2
http://www.abes.fr/entities/organizations/ppn/028968832,Alger (Période coloniale)
http://www.abes.fr/entities/organizations/ppn/030142199,Bordeaux (1441-1970)
http://www.abes.fr/entities/organizations/ppn/030142865,Clermont-Ferrand
http://www.abes.fr/entities/organizations/ppn/034751386,Ecole nationale supérieure des industries alimentaires - Massy
http://www.abes.fr/entities/organizations/ppn/030937809,Ecole des Mines de Douai
http://www.abes.fr/entities/organizations/ppn/026436930,"Ecole nationale du génie rural, des eaux et forêts"
http://www.abes.fr/entities/organizations/ppn/026523477,Ecole nationale supérieure d'agronomie de Montpellier
http://www.abes.fr/entities/organizations/ppn/029884942,Faculté de droit et sciences économiques de Grenoble
http://www.abes.fr/entities/organizations/ppn/072323965,Faculté des lettres et sciences humaines de Grenoble


### Habilitations partielles et délégations partielles non réciproques

On vérifie qu'une habilitation partielle donne lieu à une délégation partielle. Les dates d'habilitation partielle et de la délégation partielle doivent correspondre.

In [97]:
%qparam infer false
%qparam sameAs false
%show 20

PREFIX mov: <http://www.abes.fr/vocabularies/movies#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT DISTINCT * {
    ?etat a <http://www.abes.fr/entities/etats>;
          mov:from ?from;
          mov:to ?to;
          mov:start ?start;
          mov:end ?end;
          mov:statetype "delegationPartielle".

    OPTIONAL {
        [] a <http://www.abes.fr/entities/etats>;
               mov:from ?to;
               mov:start ?start;
               mov:end ?end;
               mov:statetype "habilitationPartielle". 
    }
}


Param set: infer = false
Param set: sameAs = false
Result maximum size: 20

etat,from,to,start,end
http://www.abes.fr/entities/etats/c48aaf0d8dccd4686babcaf2605d4d9c7a0894ec,http://www.abes.fr/entities/organizations/ppn/139408088,http://www.abes.fr/entities/organizations/ppn/241345251,2020-01-01,9999-12-31
http://www.abes.fr/entities/etats/d6679b452c95c205be91388b12c002d1fd00ba83,http://www.abes.fr/entities/organizations/ppn/027404978,http://www.abes.fr/entities/organizations/ppn/238277429,2020-01-01,9999-12-31
http://www.abes.fr/entities/etats/976c0d754e863a784f28c593a1b732891745a8e0,http://www.abes.fr/entities/organizations/ppn/026375060,http://www.abes.fr/entities/organizations/ppn/121855465,2008-01-01,2015-03-01


### Habilitation sans fin (alors que l'établissement a disparu)

In [98]:
%qparam infer true
%qparam sameAs false
%show 20

PREFIX mov: <http://www.abes.fr/vocabularies/movies#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT * WHERE {
    ?existence a <http://www.abes.fr/entities/etats>;
        mov:from ?from;
        mov:statetype "existence";
        mov:end ?existence_end

    OPTIONAL {
        ?habilitation mov:from ?from;
           mov:end ?habilitation_end;
           mov:statetype ?doct_state.
    }
    
    FILTER(?doct_state IN ("habilitationDoctorale", "delegationDoctorale", "habilitationPartielle","delegationPartielle"))
    FILTER(?habilitation_end = "9999-12-31"^^xsd:dateTime) 
}

Param set: infer = true
Param set: sameAs = false
Result maximum size: 20

existence,from,existence_end,habilitation,habilitation_end,doct_state
http://www.abes.fr/entities/etat/existence/RDy6Q/caaf6ecac0f4e1626742a28477d20be1,http://www.abes.fr/entities/organizations/ppn/185433669,2019-12-31T00:00:00Z,http://www.abes.fr/entities/etats/a2d89d794779631f320c79de96d071884c7fbf7d,9999-12-31,habilitationDoctorale
http://www.abes.fr/entities/etat/existence/hB4QE/e011996ef62b1f7f01b7968c7af969e3,http://www.abes.fr/entities/organizations/ppn/184668794,2019-12-31T00:00:00Z,http://www.abes.fr/entities/etats/d1c26eb0a265f6749e903dd9cc69853d9de17b6f,9999-12-31,habilitationDoctorale
http://www.abes.fr/entities/etat/existence/O2v0H/5d5175df7b9fae445998009dd456d367,http://www.abes.fr/entities/organizations/ppn/026404788,2019-12-31T00:00:00Z,http://www.abes.fr/entities/etats/25630048aee1a4e5d72bafe82efe4f59c8e0112d,9999-12-31,delegationDoctorale
http://www.abes.fr/entities/etat/existence/7CYWd/5d5175df7b9fae445998009dd456d367,http://www.abes.fr/entities/organizations/ppn/026403498,2019-12-31T00:00:00Z,http://www.abes.fr/entities/etats/87bacdbca770db68da0958c8f5bcf22f5e1e1280,9999-12-31,delegationDoctorale
http://www.abes.fr/entities/etat/existence/9Y8CD/a9d390293bdb93ffa75b0aa9ae9b937a,http://www.abes.fr/entities/organizations/ppn/030820499,2019-12-31T00:00:00Z,http://www.abes.fr/entities/etats/9777dcee316366cab5c4472ec6e5e3e51fab8833,9999-12-31,delegationDoctorale
http://www.abes.fr/entities/etat/existence/wGM55/4b04936292ede989b755e1765cba7daa,http://www.abes.fr/entities/organizations/ppn/027960250,2014-12-31T00:00:00Z,http://www.abes.fr/entities/etats/d0aeca2d0af40a8d2fcec113b4d62b382c06c403,9999-12-31,delegationDoctorale


### États et existences incompatibles

Les états sont considérés comme invalides s'ils débutent avant la période d'existence de l'établissement ou s'ils cessent après cette période (à l'exception les états de succession).

In [99]:
%qparam infer true
%qparam sameAs false
%show 20

PREFIX mov: <http://www.abes.fr/vocabularies/movies#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?state ?existence ?state_start ?state_end ?existence_start ?existence_end ?diagnostic WHERE {
    ?existence a <http://www.abes.fr/entities/etats>;
        mov:from ?from;
        mov:statetype "existence";
        mov:start ?existence_start.
    
    OPTIONAL {
        ?existence mov:end ?e_end.
    }
    
    OPTIONAL {
        ?state a <http://www.abes.fr/entities/etats>;
           mov:from ?from;
           mov:start ?state_start;
           mov:statetype ?state_type.
        
        OPTIONAL {
            ?state mov:end ?s_end.
        }
    }
    
    FILTER(?state_type NOT IN ("existence", "succession"))
    
    BIND (IF(!BOUND(?s_end), "9999-12-31"^^xsd:date, ?s_end )  AS ?state_end)
    BIND (IF(!BOUND(?e_end), "9999-12-31"^^xsd:date, ?e_end )  AS ?existence_end)
    
    BIND( IF(?state_start < ?existence_start && ?state_end > ?existence_end, "?state_start < ?existence_start && ?s_end > ?e_end", BNODE()) AS ?diagnostic)
    BIND( IF(?state_start < ?existence_start, "l'état débute avant l'existence", BNODE()) AS ?diagnostic)
    BIND( IF(?state_end > ?existence_end, "l'état termine après l'existence", BNODE()) AS ?diagnostic)

    FILTER(!ISBLANK(?diagnostic))
}

Param set: infer = true
Param set: sameAs = false
Result maximum size: 20

state,existence,state_start,state_end,existence_start,existence_end,diagnostic
http://www.abes.fr/entities/etats/729242428330ac311a2171c637b5c0bb82ed88e6,http://www.abes.fr/entities/etat/existence/p25Q3/ad293659175ee280c1924086200f917c,1432-01-01,2016-12-31,1971-01-01T00:00:00Z,9999-12-31,l'état débute avant l'existence
http://www.abes.fr/entities/etats/a106c38edcf5e702277d7b752a60b3818bd624b4,http://www.abes.fr/entities/etat/existence/atbEK/4e0d6df746322a4e1a5798a1c437eabd,1993-01-20,9999-12-31,1993-01-22T00:00:00Z,9999-12-31,l'état débute avant l'existence
http://www.abes.fr/entities/etats/78875d80c227fc8c2c3902c1ebaa3ef879fd866a,http://www.abes.fr/entities/etat/existence/7Mpgt/ad293659175ee280c1924086200f917c,1968-01-01,2016-12-31,1971-01-01T00:00:00Z,9999-12-31,l'état débute avant l'existence
http://www.abes.fr/entities/etats/91b19da797056f910532c3f747163cb004242c16,http://www.abes.fr/entities/etat/existence/ti37C/ad293659175ee280c1924086200f917c,1969,9999-12-31,1971-01-01T00:00:00Z,9999-12-31,l'état débute avant l'existence
http://www.abes.fr/entities/etats/88479ecbfa9f937034c076716d3a9b5eff895bf4,http://www.abes.fr/entities/etat/existence/j5bS4/ca3f78ee242cc8194613c11a1e07e7d7,2013-10-17,9999-12-31,2013-10-19T00:00:00Z,9999-12-31,l'état débute avant l'existence
http://www.abes.fr/entities/etats/a18e580ce18e63f5a3a1a9b414edbdbafbf01c54,http://www.abes.fr/entities/etat/existence/O2v0H/5d5175df7b9fae445998009dd456d367,1970-03-21,2014-12-31,1971-01-01T00:00:00Z,2019-12-31T00:00:00Z,l'état débute avant l'existence
http://www.abes.fr/entities/etats/d3daebe81d451ab29fc5c6d2ff05991cb729e073,http://www.abes.fr/entities/etat/existence/5KpJ9/5d5175df7b9fae445998009dd456d367,1970-03-21,2014-12-31,1971-01-01T00:00:00Z,2019-12-31T00:00:00Z,l'état débute avant l'existence
http://www.abes.fr/entities/etats/2042d585e9e059524c28b3f896e284a93a797c53,http://www.abes.fr/entities/etat/existence/kvciR/56539f629975363b2513ce5c9258f996,1970-03-21,2017-12-31,1971-01-01T00:00:00Z,2017-12-31T00:00:00Z,l'état débute avant l'existence
http://www.abes.fr/entities/etats/252c95b8587ccaf36be2cecbb48da6ac73a93a23,http://www.abes.fr/entities/etat/existence/3itX6/49bf00b8715ad00ed7ea2db749f67610,1976-01-01,2016-12-31,1976-03-18T00:00:00Z,2016-12-31T00:00:00Z,l'état débute avant l'existence
http://www.abes.fr/entities/etats/f689950e6a69ddebd3e7570545abf98432d5bb1d,http://www.abes.fr/entities/etat/existence/cqkij/ad293659175ee280c1924086200f917c,1970,9999-12-31,1971-01-01T00:00:00Z,9999-12-31,l'état débute avant l'existence


## Enrichissement des données

### Candidats enrichissement des données ABES/Paysage
On recherche les candidats non liés par leur nom (foaf:name|skos:prefLabel|skos:altLabel)

In [8]:
%qparam infer false
%qparam sameAs false

PREFIX mov: <http://www.abes.fr/vocabularies/movies#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT * WHERE {
    ?org_abes a foaf:Organization;
        foaf:name ?name.
    
    FILTER (STRSTARTS(str(?org_abes), "http://www.abes.fr/entities/organizations/ppn"))
    
    MINUS { 
        ?org_paysage (^owl:sameAs|owl:sameAs) ?org_abes .
        FILTER (STRSTARTS(str(?org_paysage), "http://paysage.mesri.fr/structures/"))
    }
    
    ?candidat_paysage (foaf:name|skos:prefLabel|skos:altLabel) ?name.
    FILTER (STRSTARTS(str(?candidat_paysage), "http://paysage.mesri.fr/structures/"))
}

Param set: infer = false
Param set: sameAs = false

org_abes,name,org_paysage,candidat_paysage
http://www.abes.fr/entities/organizations/ppn/027291340,Université de Lille,,http://paysage.mesri.fr/structures/Szqy5
http://www.abes.fr/entities/organizations/ppn/027291340,Université de Lille,,http://paysage.mesri.fr/structures/Szqy5
http://www.abes.fr/entities/organizations/ppn/027291340,Université de Lille,,http://paysage.mesri.fr/structures/Szqy5
http://www.abes.fr/entities/organizations/ppn/236453505,Université de Paris,,http://paysage.mesri.fr/structures/5cZyU
http://www.abes.fr/entities/organizations/ppn/236453505,Université de Paris,,http://paysage.mesri.fr/structures/kyS9n
http://www.abes.fr/entities/organizations/ppn/236453505,Université de Paris,,http://paysage.mesri.fr/structures/5cZyU
http://www.abes.fr/entities/organizations/ppn/236453505,Université de Paris,,http://paysage.mesri.fr/structures/kyS9n
http://www.abes.fr/entities/organizations/ppn/236453505,Université de Paris,,http://paysage.mesri.fr/structures/5cZyU
http://www.abes.fr/entities/organizations/ppn/236453505,Université de Paris,,http://paysage.mesri.fr/structures/kyS9n


## Ontologies

<center>
    <figure>
        <img src="movies_ontology_v1.svg">
        <figcaption>Ontologie movies v1</figcaption>
    </figure>
</center>

<center>
    <figure>
        <img src="movies_ontology_v2.svg">
        <figcaption>Ontologie movies v2</figcaption>
    </figure>
</center>

